# NACTO Transit Typologies

## Criteria to Use
* Create a `route_typology` and `freq_category` column.
   * Since only `rapid` and `local` route typologies are associated wiht a specific service frequency, we will want to use differentiate between `low_frequency/downtown_local` from `high_frequency/downtown_local`.
* 2 mile long road segments
* `stop_frequency` = stops per mile
* `service_frequency` = peak hours (buses per hour)
* wherever there are overlapping cutoffs, the more generous criteria is assigned to the road
* 1 shape selected for each route-direction
* shape is overlayed against buffered road segments, and then a % for each typology is calculated
* A transit route can be considered that typology as long as more than 10% of its length falls in that typology

### [Transit route types](https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-route-types/)
* Use **stop frequency (per mile) and freq cateogry from below** for scoring
* Downtown local: stop frequency 4 or more per mile, service area is compact and dense (how do we measure?)
* Local: stop frequency 3-4 per mile, service frequency is moderate or high, riders use for short-to-medium length trips (how do we measure?)
* Rapid: stop frequency 1-3 per mile, service frequency is moderate or high
* Coverage: stop frequency is 2-8 per mile, service frequency is low, service area is low density
* Express: non-stop express segments between service areas with more frequent stops, service frequency is scheduled, often infrequent, and concentrated during peak
 
### [Transit frequency and volume](https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-frequency-volume/)
* Use **x buses per hour** for scoring
* Low volume: over 15 min headways = 4 or fewer buses per hour, fewer than 100 passengers per hour
* Moderate volume: 10-15 min headways, 4-10 buses per hour, 100-750 passengers per hour
* High volume: 2-6 minute combined headways, 10-30 buses per hour, 500-2_000 passengers per hour
* Very high volume: 2-3 minute combined headways, 20-30 buses per hour or more, 1_000+ passengers per hour on multiple routes or 2_500+ per hour on one route with multi-unit vehicles.

In [1]:
import altair as alt
import geopandas as gpd
import pandas as pd
       
from shared_utils import rt_dates
from route_typologies import prep_roads, overlay_shapes_to_roads
analysis_date = rt_dates.DATES["jan2024"]

In [2]:
road_stats = prep_roads()

In [3]:
display(road_stats.freq_category.value_counts(dropna=False))
display(road_stats.freq_category.value_counts(dropna=False, normalize=True))

low          41951
moderate     14708
high          6192
very_high     6158
Name: freq_category, dtype: int64

low          0.607906
moderate     0.213132
high         0.089727
very_high    0.089235
Name: freq_category, dtype: float64

In [4]:
display(road_stats.typology.value_counts(dropna=False))
display(road_stats.typology.value_counts(dropna=False, normalize=True))

coverage          37728
downtown_local    13322
rapid             10082
None               4734
local              3143
Name: typology, dtype: int64

coverage          0.546711
downtown_local    0.193047
rapid             0.146097
None              0.068600
local             0.045545
Name: typology, dtype: float64

In [5]:
pd.crosstab(road_stats.typology, road_stats.freq_category)

freq_category,high,low,moderate,very_high
typology,,,,
coverage,0,37728,0,0
downtown_local,2022,4223,3211,3866
local,1102,0,2041,0
rapid,2729,0,7353,0


In [6]:
pd.crosstab(road_stats.typology, road_stats.freq_category, normalize=True)

freq_category,high,low,moderate,very_high
typology,,,,
coverage,0.000000,0.586978,0.000000,0.000000
downtown_local,0.031459,0.065702,0.049957,0.060148
local,0.017145,0.000000,0.031754,0.000000
rapid,0.042458,0.000000,0.114399,0.000000


In [7]:
df = (road_stats.groupby(["freq_category", "typology"])
      .agg({"linearid": "count"})
      .reset_index()
      .rename(columns = {"linearid": "counts"})
     )

In [8]:
alt.renderers.enable("html")
alt.data_transformers.enable('default', max_rows=None)

chart = (alt.Chart(df)
 .mark_rect()
 .encode(
    x=alt.X(
        'freq_category:O', 
        sort=["low", "moderate", "high", "very_high"]
    ),
    y=alt.Y(
        'typology:O', 
        sort=["downtown_local", "local", "rapid", "coverage"]
    ),
    color=alt.Color('counts:Q', scale=alt.Scale()),
    tooltip = ["freq_category", "typology", "counts"]
).properties(
     width=200, height=200, 
     title="Service Frequency vs Route Typology"
 )
)

chart

alt.Chart(...)

In [9]:
results = overlay_shapes_to_roads(
    road_stats,
    analysis_date,
    buffer_meters=20
)

In [10]:
# Get counts by the combinations of freq_category-typology
# the heatmap for the transit routes categorized
results2 = results.groupby(
    ["freq_category", "typology"]
).agg({"route_id": "count"}).reset_index()

In [11]:
results2.sort_values(["typology", "freq_category"])

,freq_category,typology,route_id
3,low,coverage,3422
0,high,downtown_local,2277
4,low,downtown_local,1854
5,moderate,downtown_local,2274
8,very_high,downtown_local,2676
1,high,local,2138
6,moderate,local,2145
2,high,rapid,2886
7,moderate,rapid,3315


In [12]:
results.groupby(["typology"]
).agg({"route_id": "count"}).reset_index()

,typology,route_id
0,coverage,3422
1,downtown_local,9081
2,local,4283
3,rapid,6201


In [13]:
# look at values of pct_typology for every segment
# decide where "significant" cutoff is
# since we don't want to display every variation possible for a route
results.pct_typology.describe()

count    22987.000000
mean         0.132573
std          0.227942
min          0.000000
25%          0.010000
50%          0.040000
75%          0.150000
max          3.190000
Name: pct_typology, dtype: float64

In [14]:
results[results.pct_typology > 1].shape, results.shape

((326, 9), (22987, 9))

In [17]:
alt.renderers.enable("html")
alt.data_transformers.enable('default', max_rows=None)

chart2 = (alt.Chart(results2)
 .mark_rect()
 .encode(
    x=alt.X(
        'freq_category:O', 
        sort=["low", "moderate", "high", "very_high"]
    ),
    y=alt.Y(
        'typology:O', 
        sort=["downtown_local", "local", "rapid", "coverage"]
    ),
    color=alt.Color('route_id:Q', scale=alt.Scale()),
    tooltip = ["freq_category", "typology", "route_id"]
).properties(
     width=200, height=200, 
     title="Transit Service Frequency vs Route Typology"
 )
)

chart2

alt.Chart(...)

Only keep route typologies if over 10% of the route length is that.

* Since roads run in 2 directions, this value can be over 100% 
(though only ~300ish rows out of 22-24k >1).
* median is 4%, 75th percentile is ~15%. 
* Given that there are 8 possible combos, we'll pick threshold of 10% so we can see significant typologies